# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from VAE1D import *

In [3]:
from IPython.core.debugger import set_trace

Define train function in notebook due to import namespace bug.

In [25]:
def train_VAE1D(dl):
    """
    Execute the training loop
    """
    loss_tracker = AvgTracker()
    kl_tracker = AvgTracker()
    logp_tracker = AvgTracker()
    timer = StopWatch()
    freq = min(log_freq, len(dl))
    
    for i, (X, _) in enumerate(tqdm(dl)):
        
        X = X.to(device)
        timer.lap()  # load time
        
        # Generate transient and compute loss
        X_hat, mu, logvar = model(X)
        loss, loss_desc = criterion(X_hat, X, mu, logvar)
        timer.lap()  # gen time
        
        loss_tracker.update(loss.item())
        kl_tracker.update(loss_desc['KL'].item())
        logp_tracker.update(loss_desc['logp'].item())
        
        if model.training:
            # Update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            timer.lap()  # backprop time
        
        if ((i + 1) % freq == 0) and ((epoch + 1) % ep_freq == 0):
            # Print progress
            if not model.training:
                print('VALIDATION')
            print(f'Epoch: {epoch + 1} ({i + 1}/{len(dl)})')
            print(f'\tData load time: {timer.elapsed[0]:.3f} sec')
            print(f'\tGeneration time: {timer.elapsed[1]:.3f} sec')
            if model.training:
                print(f'\tBackprop time: {timer.elapsed[2]:.3f} sec')
            print(f'\tLog probability: {logp_tracker.val:.4f} '
                  f'(avg {logp_tracker.avg:.4f})')
            print(f'\tKL: {kl_tracker.val:.4f} (avg {kl_tracker.avg:.4f})')
            print(f'\tLoss: {loss_tracker.val:.4f} (avg {loss_tracker.avg:.4f})')

    return loss_tracker.avg, kl_tracker.avg, logp_tracker.avg

# Accumulator

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [5]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [6]:
model = VAE1D(size, n_channels, n_latent)

In [7]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [8]:
desc = 'accumulator'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [9]:
print(len(train_dl), len(val_dl), len(test_dl))

27 7 1157


### Prepare for training

In [10]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-accumulator already exists


PosixPath('models/190130-accumulator')

In [11]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [12]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [13]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [14]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')

 74%|███████▍  | 20/27 [00:01<00:00, 18.45it/s]

Epoch: 1 (15/27)
	Data load time: 0.128 sec
	Generation time: 0.156 sec
	Backprop time: 0.181 sec
	Log probability: -643.5981 (avg -943.6181)
	KL: 228.5415 (avg 125734.2011)
	Loss: 2929.0132 (avg 1258285.6947)


100%|██████████| 7/7 [00:00<00:00, 26.26it/s]


Validation
Epoch: 1 (7/7)
	Data load time: 0.126 sec
	Generation time: 0.133 sec
	Log probability: -547.6620 (avg -537.7210)
	KL: 140.9746 (avg 143.3336)
	Loss: 1957.4082 (avg 1971.0570)
Saving checkpoint..


  0%|          | 0/27 [00:00<?, ?it/s]

...

Validation
Epoch: 299 (7/7)
	Data load time: 0.137 sec
	Generation time: 0.145 sec
	Log probability: -5.8602 (avg -7.1443)
	KL: 7.8615 (avg 7.7725)
	Loss: 84.4752 (avg 84.8694)
Lowest validation loss: 39.3230


 70%|███████   | 19/27 [00:00<00:00, 17.78it/s]

Epoch: 300 (15/27)
	Data load time: 0.146 sec
	Generation time: 0.163 sec
	Backprop time: 0.190 sec
	Log probability: -11.5084 (avg -19.1697)
	KL: 6.0360 (avg 6.9560)
	Loss: 71.8686 (avg 88.7295)


100%|██████████| 7/7 [00:00<00:00, 25.87it/s]

Validation
Epoch: 300 (7/7)
	Data load time: 0.129 sec
	Generation time: 0.136 sec
	Log probability: -9.9763 (avg -13.4556)
	KL: 26.1172 (avg 20.0997)
	Loss: 271.1487 (avg 214.4525)
Lowest validation loss: 39.3230


# Cooler

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [26]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [27]:
model = VAE1D(size, n_channels, n_latent)

In [28]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [29]:
desc = 'cooler'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [30]:
print(len(train_dl), len(val_dl), len(test_dl))

28 7 1100


### Prepare for training

In [31]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-cooler already exists


PosixPath('models/190130-cooler')

In [32]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [33]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [34]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [35]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')


100%|██████████| 28/28 [00:01<00:00, 22.67it/s]

100%|██████████| 7/7 [00:00<00:00,  9.88it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 2679.8072



100%|██████████| 28/28 [00:01<00:00, 23.19it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 960.9481



100%|██████████| 28/28 [00:01<00:00, 23.33it/s]

100%|██████████| 7/7 [00:00<00:00,  9.51it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

...

VALIDATION
Epoch: 290 (7/7)
	Data load time: 0.148 sec
	Generation time: 0.154 sec
	Log probability: -2.1864 (avg -1.7038)
	KL: 21.2849 (avg 23.0191)
	Loss: 215.0359 (avg 231.8952)



100%|██████████| 28/28 [00:01<00:00, 22.50it/s]

100%|██████████| 7/7 [00:00<00:00,  8.18it/s]

100%|██████████| 28/28 [00:01<00:00, 22.75it/s]

100%|██████████| 28/28 [00:01<00:00, 22.60it/s]

100%|██████████| 7/7 [00:00<00:00,  8.33it/s]

100%|██████████| 28/28 [00:01<00:00, 22.54it/s]

100%|██████████| 7/7 [00:00<00:00,  8.37it/s]

100%|██████████| 28/28 [00:01<00:00, 22.57it/s]

100%|██████████| 7/7 [00:00<00:00,  8.20it/s]

100%|██████████| 28/28 [00:01<00:00, 22.57it/s]

100%|██████████| 7/7 [00:00<00:00,  8.75it/s]

100%|██████████| 28/28 [00:01<00:00, 22.57it/s]

100%|██████████| 7/7 [00:00<00:00,  8.38it/s]

100%|██████████| 28/28 [00:01<00:00, 22.84it/s]

100%|██████████| 28/28 [00:01<00:00, 22.50it/s]

100%|██████████| 7/7 [00:00<00:00,  8.62it/s]

 68%|██████▊   | 19/28 [00:00<00:00, 18.12it/s]

Epoch: 300 (15/28)
	Data load time: 0.147 sec
	Generation time: 0.159 sec
	Backprop time: 0.179 sec
	Log probability: -31.5539 (avg -15.3450)
	KL: 27.6957 (avg 29.7413)
	Loss: 308.5110 (avg 312.7578)



100%|██████████| 28/28 [00:01<00:00, 22.62it/s]

100%|██████████| 7/7 [00:00<00:00,  8.68it/s]


VALIDATION
Epoch: 300 (7/7)
	Data load time: 0.142 sec
	Generation time: 0.149 sec
	Log probability: -8.5994 (avg -9.0300)
	KL: 24.9412 (avg 25.1332)
	Loss: 258.0119 (avg 260.3620)


# Pump

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [51]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [52]:
model = VAE1D(size, n_channels, n_latent)

In [53]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [54]:
desc = 'pump'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [55]:
print(len(train_dl), len(val_dl), len(test_dl))

33 9 920


### Prepare for training

In [56]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

Folder models/190130-pump already exists


PosixPath('models/190130-pump')

In [57]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [58]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [59]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [60]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')


  0%|          | 0/33 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 1778.9485



  0%|          | 0/33 [00:00<?, ?it/s]

Saving checkpoint..
Lowest validation loss: 1534.1309



  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch: 300 (15/33)
	Data load time: 0.157 sec
	Generation time: 0.169 sec
	Backprop time: 0.189 sec
	Log probability: -10.4840 (avg -15.6277)
	KL: 12.8061 (avg 14.0020)
	Loss: 138.5450 (avg 155.6473)



  0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 300 (30/33)
	Data load time: 0.157 sec
	Generation time: 0.169 sec
	Backprop time: 0.189 sec
	Log probability: -6.8547 (avg -14.7242)
	KL: 13.5234 (avg 13.2518)
	Loss: 142.0882 (avg 147.2424)



100%|██████████| 9/9 [00:00<00:00, 27.16it/s]

VALIDATION
Epoch: 300 (9/9)
	Data load time: 0.156 sec
	Generation time: 0.164 sec
	Log probability: -52.4102 (avg -17.7443)
	KL: 6.1565 (avg 14.4458)
	Loss: 113.9753 (avg 162.2020)


# Valve

### Build model and loss function 
Initially designed for 2D input images, modified for 1D time-series data.
Based on this paper: https://arxiv.org/abs/1807.01349

In [ ]:
# The hydraulic system has 14 sensors from which to pull data
n_channels = 14
# The data has been resized to 512, although it represents 1 min for each cycle
size = 512
# Latent space is restricted to be about 1/170th of the input dims, similar to 2D case
n_latent = 50

In [ ]:
model = VAE1D(size, n_channels, n_latent)

In [ ]:
beta = 10  # KL term relative weight
criterion = VAE1DLoss(beta)

### Load hydraulic test data
From this dataset: https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems#

In [ ]:
desc = 'valve'
data_path = Path(f'data/hydraulic/{desc}/')
batch_size = 32
train_dl, val_dl, test_dl = load_datasets(data_path,
                                          batch_size=batch_size)

In [ ]:
print(len(train_dl), len(val_dl), len(test_dl))

### Prepare for training

In [ ]:
# Display settings - TODO add Visdom logging
log_freq = 15 # batches
ep_freq = 10  # epochs

# Training parameters
epochs = 300
lr = 1e-2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint to resume from (default None)
load_path = None

# Checkpoint save location
save_path = Path(f"models/{date.today().strftime('%y%m%d')}-{desc}/")
if save_path.is_dir():
    print(f"Folder {save_path} already exists")
else:
    os.mkdir(save_path)
save_path

In [ ]:
# Load checkpoint if any
if load_path is not None:
    checkpoint = torch.load(load_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print("Checkpoint loaded")
    print(f"Validation loss: {checkpoint['val_loss']}")
    print(f"Epoch: {checkpoint['epoch']}")

In [ ]:
# Load optimizer and scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [ ]:
# Move to GPU
model = model.to(device)
criterion = criterion.to(device)

### Train the model

In [ ]:
# Main loop
best_loss = np.inf
for epoch in range(epochs):

    model.train()
    scheduler.step()
    train_loss, train_kl, train_logp = train_VAE1D(train_dl)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_kl, val_logp = train_VAE1D(val_dl)

    # Report training progress to user
    if val_loss < best_loss:
        print('Saving checkpoint..')
        best_loss = val_loss
        save_dict = {'epoch': epoch + 1,
                     'state_dict': model.state_dict(),
                     'val_loss': val_loss,
                     'optimizer': optimizer.state_dict()}
        path = save_path / f'best_model-{n_latent}.pt'
        torch.save(save_dict, path)
        print(f'Lowest validation loss: {best_loss:.4f}')